<center>
    <h1>Modeling an Electron in a Field of Secured Point Charges</h1>
</center>

<h2>Introduction</h2>

Kinetics typically makes sense. You can see kinetics. But, you can't really see an electric field. The idea that an electron will move towards a positive charge and away from a negative charge might make sense, but it can be difficult to visualize where an electron might go in a more complicated field of charges. This project is about helping students in physics classes understand how electrons are moved in electric fields

In my project, we are looking at an idealized simulation. The only forces present on the electron are the forces created by the charges, and those charges are all point charges fixed in place that won't be affected by each other. While not an extrememly realistic situation, it is a good situation for beginners to get a better understanding.

The simulation relies on Coulomb's Law, which is used to calculate the force on a point charge by another point charge. It also uses Newton's Second Law (f=ma) to get acceleration from force. Finally, we use Reimann sums, as velocity is the area under the curve of acceleration and position the area under the curve of velocity.

<h2>Methodology</h2>

<h3>Imports and Libraries</h3>

<p>The first cell in any coding project is typically for import statements, and this project is not an exception to that rule. Modeling requires much more than the basic Python, and all of the functions, objects, and constants that I need to make this project work are imported here.</p>
<p>This cell wasn't written all at once. I'd go as far as to say that this cell was the one I wrote last because while I did most of the other work in order, I was consistently coming back to this cell and importing one more thing as I learned something else could be useful or that I needed one more thing to fix an error.</p>
<p>Being able to easily import things from various libraries is one of the best parts about Python, and all of these things were needed and will be explained later in the context of my code.</p>

In [1]:
import math #basic python math library, used for square rooting and getting pi
#https://docs.python.org/3/library/math.html#
import numpy as np #common library, source of numpy arrays which are easier to manipulate as vectors than your average python list
#https://numpy.org/doc/stable/index.html

from scipy.constants import e, m_e, epsilon_0 
#physics constants I need, the charge of a proton or electron (e), the mass of an electron (m_e), and epsilon naught
#https://docs.scipy.org/doc/scipy/reference/constants.html

import matplotlib.pyplot as plt #for plotting data
#https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
import matplotlib.animation as animation #to create animation
#https://matplotlib.org/stable/api/animation_api.html
from IPython.display import HTML #making it possible to display my animation
#https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

<h3>Basic Objects (Charge)</h3>

<p>The purpose of the Charge Object is to be able to make variables that can hold all of the parts of a charge. Simple variables, like ints and strings, can only hold one piece of information, and while lists can hold multiple pieces of information, they aren't consistent. All charges have the same types of information that they need to hold, so creating an Object is the simplest way to standardize charges.</p>
<p>The first piece of information that a charge has is its position, where it is located on the graph. Since this model is only 2D, the position has only two parts, the x element and the y element. This could be held in a list or as two seperate floats, but I decided to have position be an ndarray for the abilities that ndarrays have for math. ndarray's function similarly to vectors, and can be added or subtracted while lists can't be. When two lists are added together, they get concatenated, but when two ndarrays of the same length are added or subtracted, a new ndarray is created where the elements in each ndarray are added or subtracted, which is useful since I'll need to find the distance between charges and the electron in the future (<em>The N-dimensional</em>, n.d.).</p>
<p>The second piece of information is the charge of the charge, which can be any positive or negative float. There really isn't anything complicated about the charge of a charge, but it's important to know.</p>
<p>There a three methods (functions for objects are known as methods in Python) that a Charge Object has. The first every object needs, and that is an initialization method. It has two underscores in front of and behind its name because it is what is known as a magic method. Magic methods are different than normal methods because they aren't meant to be called directly. Since an object doesn't exist until initialization, it has to be a magic method because you can't call a method on an object that doesn't exist yet. Beyond initialization, there are two methods for a Charge Object known as getter methods. The purpose of these methods is to "get" the variable that exist within an object. Here, there's a getter method for position and a getter method for charge. All these do is return the value of the position or charge of a Charge Object.</p>

In [2]:
class Charge: #class to create Charge objects, which will make it easier to work with charges

    def __init__(self, x, y, charge): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) ##creating the position, using the given x and y coordinates, as an array
        #https://numpy.org/doc/stable/reference/generated/numpy.array.html
        self.charge = charge #getting the charge of the particle

    def get_position(self): #function to get the position of a charge
        return self.position #return the position

    def get_charge(self): #function to get the charge of a charge
        return self.charge #return the charge

<h3>Complicated Objects (Electron)</h3>

<p>Like for charges, I made an Electron Object to be able to hold all of the different parts of an electron. Unlike a charge, where everything is held constant throughout the simulation, an electron has a lot of things that are happening to it, and so needs more variable and more methods.</p>
<p>First of all, there's more variables. Electron's also have position and charge (though all of them have a charge of -e), but since they are moving they also have a velocity and acceleration. These are both initially [0, 0] (using ndarrays because velocity and acceleration are both 2D vectors in a 2D space), as before the model starts the electron has no forces on it and does not start out moving.</p>
<p>Electron's also have getter methods, but they only have them for position, velocity, and acceleration, because it can be assumed that every electron has a charge of -e so it is unnecessary to have a way to access the charge of the electron outside of the Electron Object.</p>
<p>The most important method that an electron has is <em>get_force(self, charges)</em>. This method calculates the force on the electron at it's current position. It takes in a list of all of the charges the are on the graph. Starting with force=[0, 0], the force of each individual charge on the electron is calculated and added to the force to end up with the total force. The force is calculated with Coulmob's Law:</p>
$$
    F=\frac{q_1*q_2}{4\pi*e_0}*\frac{r_1-r_2}{|r_1-r_2|^3}
$$
<p>Once the total force is calculated, acceleration can be found using Newton's Second Law ($F=ma$). For the purposes of the model, I slightly adjust the acceleration by dividing by an additional 100 so the acceleration is smaller than found. This is because using the actual acceleration results in the model failing due to consistent overshooting of position. Since force is inversely proportional to the square of the distance between the electron and a charge, once the electron gets too far away it will never course correct back to where it should end up. As large movements make it easy for the model to mess up, I've added in the adjustment to increase accuracy.</p>
<p>Finally, there are methods to update the velocity and the position. Velocity is the area under the curve of acceleration, so when integration isn't desirable, velocity can be found through adding up all of the accelerations from start to finish. So, whenever velocity needs to be updated, acceleration is added to the current velocity. This same process works for position, as position is the area under the curve of velocity.</p>

In [3]:
class Electron: #class to create Electron objects, which will allow me to make functions for electrons

    def __init__(self, x, y): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) #creating the position, using the given x and y coordinates, as an array
        self.charge = -e #all electrons have a charge of -e
        self.velocity = np.array([0.0, 0.0]) #start with 0 velocity
        self.acceleration = np.array([0.0, 0.0]) #start with 0 acceleration

    def get_position(self): #function to get the position of an electron
        return self.position #return the position

    def get_velocity(self): #function to get the velocity of an electron
        return self.velocity #return the velocity
    
    def get_acceleration(self): #function to get the acceleration of an electron
        return self.acceleration #return acceleration

    def get_force(self, charges): #function to get the sum of all the forces on an electron
        force = np.array([0.0, 0.0]) #start with 0
        for charge in charges: #then go through each charge
            r = math.sqrt((charge.get_position() - self.get_position())[0] ** 2 + (charge.get_position() - self.get_position())[1] ** 2)
            #get the distance from electron to charge
            force += (self.get_position() - charge.get_position()) * charge.get_charge() * self.charge / 4 / math.pi / epsilon_0 / (r ** 3)
            #use Coulomb's Law to find the force a single charge places on the electron and add it to the current force
        self.acceleration = force / m_e / 100 #Newton's second law, f=ma, so a=f/m

    def update_velocity(self): #changing velocity
        self.velocity += self.acceleration * .1 #small increments, velocity changes by acceleration

    def update_position(self): #changing position
        self.position += self.velocity * .1 #small increments, position changes by velocity

<h3>Repeated Function</h3>

<p>When you need to do the same thing over and over again, you make a function to do it so that you don't have to write the same couple of lines over and over again. While ultimately <em>electron_update(charges, electron)</em> only ended up getting called once in a group, these three things should always happen in succession, so it works out for making the code easier to understand.</p>
<p>In order to update the electron for a new frame, three things need to happen. The first is finding the current force so acceleration can be found. Next, using that acceleration to get the new velocity. Then, that velocity is used to determine the next position for the electron. This function make it easy to update the electron because instead of calling all three of those things it the right order every time, I can just call <em>electron_update(charges, electron)</em>.</p>

In [4]:
def electron_update(charges, electron): #function to update the state of the electron

    electron.get_force(charges) #getting the current force
    electron.update_velocity() #updating velocity
    electron.update_position() #updating position

<h3>Generator Function</h3>

<p>Generator functions have multiple purposes, but for my model, I need it in order to control my model. There are two main methods for determining how many frames an animation will run. The first is to hard code a number of frames, but that's not going to work for this model. The second method is to use a generator function. With a generator function, the animation will run until it stops yielding (returning) numbers (ImportanceOfBeingErnest, 2018). Since I want the model to stop updating and be complete under certain circumstances, a generator function is the best way for me to determine how many frames I want the model to run for.</p>
<p>There's three possible stop conditions for when I want the model to end. The first is if the elecetron is touching a charge, which should only happen with a positive charge. Since the distance between charges will be so small at that point, the force of that one charge will overwhelm every other force and the electron will stay on that charge. My code can't understand that though, and would slingshot my electron out if I didn't stop the model. So, if the electron tooks a charge, the animation will stop. There's some range allowed because it's highly unlikely that the electron will ever have the exact same position as a charge.</p>
<p>The next condition is if the electron leaves the boundaries of my graph. I've made the assumption that since force is inversely proportional to distance squared, once an electron leaves the boundaries of my graph it is never coming back because acceleration will never (or at least, never in a reasonable amount of time) be able to turn the velocity back around.</p>
<p>Lastly, if both acceleration and velocity are [0, 0], the electron will not move again. Under those circumstances, it makes the most sense to stop modeling, as nothing will change from that point.</p>
<p>If none of these conditions are met, that means the model is not complete and another frame should be generated, so I yield the frame count (i) to let FuncAnimation know that it should continue.</p>

In [5]:
def gen(charges, electron): #making a generator function!
    i = 1 #start with one frame
    sep = True #boolean to determine if you should continue
    while sep:
        #print(i)
        for charge in charges: #look at each charge
            if abs(electron.get_position()[0] - charge.get_position()[0]) < 2 and abs(electron.get_position()[1] - charge.get_position()[1]) < 2:
            #if the electron is practically on the charge
                sep = False #don't continue
                print("on charge") #let us know we are on a charge
                print(charge.get_position(), electron.get_position()) #print both locations
                print(abs(electron.get_position()[0] - charge.get_position()[0]), abs(electron.get_position()[1] - charge.get_position()[1])) #print distance
                break #exit this loop
        if abs(electron.get_position()[0]) > 100 or abs(electron.get_position()[1]) > 100: #if it's outside of the boundaries of the simulation
            print("left boundaries")
            sep = False #don't continue
        if np.array_equal(electron.get_velocity(), [0.0,0.0]) and np.array_equal(electron.get_acceleration(), [0.0, 0.0]): #if the velocity + acceleration is 0
            print("no velocity/acceleration") #let us know that the velocity/acceleration is 0
            print(electron.get_velocity(), electron.get_acceleration()) #prove it
            sep = False #don't continue
        if sep == True: #if we still should continue
            i += 1 #up the frame number
            #print(i)
            yield i #iteration version of return

<h3>Modeling</h3>

<p>In order to make the main body of code more organized, I made a function to create the model, despite it only happening once. The function just takes in the charges and the electron so it can access all of them when creating the model.</p>
<p>The first thing I needed to go was set up the original graph. The first thing you always need to do when plotting is create a figure (typically fig) and axes (typically ax). The figure is where the graph is being drawn, while the axes are each individual subplot on the figure. Since I only need one plot for the model, everything just happens on the one set of axes (<em>Matplotlib.pyplot.subplots</em>, n.d.). Next, I plot the position of all of the charges and the starting position for the electron. I plot the position of the positive charges and negative charges seperately so that they can be different colors, making it easier for users to undertand the model. The way I do this is by creating a list by indexing through all of the elements of the charges list, and adding it to the list if it the type of charge I want. Then, since I decided that my graph would be (-100, 100) by (-100, 100), I set those to be the limits for the x and y coordinates. Finally, I made the legend visible so users would know which colors indicated which type of objects.</p>
<p>The next part is the update function, which is used for updating the animation for the model. Update functions for FuncAnimation are required to take "frame" as a parameter, so it does so (<em>Matplotlib.animation.FuncAnimation</em>, n.d.). When it's time to update to the next frame, this function is called. All it does is call <em>electron_update(charges, electron)</em>, update the position of the electron on the scatter plot by using <em>.set_offsets()</em>, which will move points on a scatter plot. Finally, it return the scatter plot so that the updated one is the one now in use.</p>
<p>After that is creating the animation. Before the animation is created, I call <em>electron.get_force(charge)</em> so that the initial acceleration isn't [0, 0] and the model immediately stops. Once that is done, the animation is created with FuncAnimation. There's a couple of parts to that. First, it needs a figure where the animation is happening, and that's fig. Then, it needs a function that will be used for getting the next frame in the animation, that's my update function. Next is frames, which gets how many frames there are, this is where the generator function from before comes into use. After that is interval, which is set to 10, so when the animation runs there's 10 milliseconds between when frames switch. Finally, save_count is a backup for if the generator fails.</p>
<p>Once the animation is created, I close the plot so it doesn't immediately pop up, as only one frame will pop up if the plot is shown without pushing it to the frontend. That's why what is return is <em>HTML(anim.to_jshtml())</em>, which basically turns the animation into something that can be read through HTML and shown on the frontend, where the user is.</p>

In [6]:
def create_animation(charges, electron): #creating the animation

    fig, ax = plt.subplots() #needed elements of the plot
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
    
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() > 0], [c.get_position()[1] for c in charges if c.get_charge() > 0], c="r", label="+ charges")
    #placing all of the positive charges on the plot, coloring them red
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.scatter.html#matplotlib.axes.Axes.scatter
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() < 0], [c.get_position()[1] for c in charges if c.get_charge() < 0 ], c="b", label="- charges")
    #placing all of the negative charges on the plot, coloring them blue
    scatter = ax.scatter(electron.get_position()[0], electron.get_position()[1], c="g", label="electron")
    #place the electron on the plot, coloring it green

    ax.set_xlim([-100, 100]) #making the x-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_xlim.html
    ax.set_ylim([-100, 100]) #making the y-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylim.html
    ax.legend() #show the legend
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.legend.html
    
    def update(frame): #update function!

        electron_update(charges, electron) #call the update function to set the values
        scatter.set_offsets(electron.get_position()) #update the position of the electron
        #https://matplotlib.org/stable/api/collections_api.html#matplotlib.collections.AsteriskPolygonCollection.set_offsets
        return scatter #return the new scatter plot

    electron.get_force(charges) #not have starting acceleration 0
    anim = animation.FuncAnimation(fig=fig, func=update, frames=gen(charges, electron), interval=10, save_count=1000) #create the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html
    plt.close() #close the plot

    return HTML(anim.to_jshtml()) #return the HTML to make the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.Animation.html#matplotlib.animation.Animation.to_jshtml

<h3>The Program</h3>

<p>This is the main body of code, while the rest of the cells were setting up things needed to make the model work, this is where everything is being pulled together. There's four main elements, the introduction, creating the charges, creating the electron, and finally, creating the model.</p>
<p>First is the introduction. This is just a couple of simple print statement, where what the model does and its boundaries are.</p>
<p>Second is creating the charges. The first thing that happens is asking how many charges a user wants to have, then turning that input into a number. That number is used as the range for a loop, where each time the loop goes through, another charge is created. In this loop, x position, y position, and charge are all asked for. Taking the information given, a new Charge Object is created and added to a list of charges for that model.</p>
<p>Next is creating an electron. All that is needed for that is getting the x and y position, which is done through input functions.</p>
<p>Lastly, the model is created by calling <em>create_animation(charges, electron)</em>.</p>

In [ ]:
print("This program will create a 2d simulation with point charges fixed in place and an unsecured electron.")
print("The dimensions are (-100, 100) by (-100, 100).")
print()

print("First, create your Charges.")
q = int(input("Number of Charges (int): "))
charges = []
for i in range(q):
    x = float(input("Enter x coordinate (float): "))
    y = float(input("Enter y coordinate (float): "))
    c = float(input("Enter charge (float): "))
    charges.append(Charge(x, y, c * e))
    
print("Second, locate your Electron.")
x = float(input("Enter x coordinate (float): "))
y = float(input("Enter y coordinate (float): "))
electron = Electron(x, y)

create_animation(charges, electron)

<h2>Results and Discussion</h2>

<h2>Conclusion</h2>

<h2>Citations</h2>

<p>
    ImportanceOfBeingErnest. (2018, February 1). <em>How to stop FuncAnimation by func
     in matplotlib? </em>[Online forum post]. Stack Overflow.
     https://stackoverflow.com/questions/48564181/
     how-to-stop-funcanimation-by-func-in-matplotlib
</p>
<p>
    <em>Matplotlib.animation.FuncAnimation</em>. (n.d.). Matplotlib. Retrieved May 9, 2024,
     from https://matplotlib.org/stable/api/_as_gen/
     matplotlib.animation.FuncAnimation.html
</p>
<p>
    <em>Matplotlib.animation.FuncAnimation</em>. (n.d.). Matplotlib. Retrieved May 9, 2024,
     from https://matplotlib.org/stable/api/_as_gen/
     matplotlib.animation.FuncAnimation.html
</p>
<p>
    <em>The n-dimensional array (ndarray)</em>. (n.d.). NumPy. Retrieved May 9, 2024, from
     https://numpy.org/doc/stable/reference/arrays.ndarray.html
</p>